In [1]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator import SyntheticGraphType

# 1. Graph generation

### Synthetic graphs
There is `SimpleSynthetic` class that should be used for synthetic graph generation.
It has enough methods to generate whatever you want to give to scheduler.

In [2]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

In [3]:
# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

wg = ss.work_graph(mode=SyntheticGraphType.GENERAL,
                   cluster_counts=10,
                   bottom_border=100,
                   top_border=200)
wg.vertex_count

180

# StructureEstimator creation

In [9]:
from random import Random
from sampo.schemas.structure_estimator import DefaultStructureEstimator, DefaultStructureGenerationEstimator

rand = Random(r_seed)
generator = DefaultStructureGenerationEstimator(rand)

sub_works = [f'Sub-work {i}' for i in range(5)]

# assign 5 uniform-distributed generations to each non-service node 
for node in wg.nodes:
    if node.work_unit.is_service_unit:
        continue
    for sub_work in sub_works:
        generator.set_probability(parent=node.work_unit.name, child=sub_work, probability=1 / len(sub_works))

structure_estimator = DefaultStructureEstimator(generator, rand) 

In [14]:
from sampo.pipeline.preparation import PreparationPipeline

restructed_wg = PreparationPipeline() \
    .wg(wg) \
    .structure_estimator(structure_estimator) \
    .build_wg()
restructed_wg.vertex_count

357